In [1]:
#%%
import numpy as np
import qiskit_metal as metal
from qiskit_metal import designs
from qiskit_metal import MetalGUI, Dict, open_docs
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.framed_path import RouteFramed
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee


from qiskit_metal.analyses import EPRanalysis

design = designs.DesignPlanar()
design.overwrite_enabled = True

design.chips.main.size['size_x'] = '7mm'
design.chips.main.size['size_y'] = '7mm'
design.chips.main.size['size_z'] = '-280um'

gui = MetalGUI(design)

RES_WIDTH = '20um'
RES_GAP = '20um'
LINE_50_OHM_WIDTH = '15um'
LINE_50_OHM_GAP = '10um'

In [2]:
"""ESTIMATE LENGTH"""
eps_eff = 6.44 # assuming eps_r = 11.9
vp = 3e8/np.sqrt(eps_eff)

def freq2length(f, vp=vp):
    return int(vp/f*1e6/4) #length of lbda/4 resonator

def length2freq(l, vp=vp):
    return int(vp/4/(l*1e-6))

l = freq2length(5e9)
print(l)
f = print(length2freq(l))

5910
5000700547


In [3]:
"""CREATE RESONATORS"""

def resonator(ID, f, tee_pos, coupling_length, prime_length, coupling_space, short_pos, res_gap, down_length = 100, fillet = 60,  right = False, orientation = '90'):
    
    if right:
        mirror = False
    else:
        mirror = True
    tee_options = Dict(prime_width=LINE_50_OHM_WIDTH, prime_gap=LINE_50_OHM_GAP, prime_length_1=str(prime_length)+'um', prime_length_2=str(prime_length)+'um',
                    second_width=RES_WIDTH, second_gap=res_gap, coupling_space=str(coupling_space)+'um', coupling_length=str(coupling_length)+'um', 
                    down_length=str(down_length)+'um', fillet=str(fillet)+'um', mirror=mirror, open_termination=True,
                    pos_x=str(tee_pos[0])+'um', pos_y=str(tee_pos[1])+'um', orientation=orientation)
    tee1 = CoupledLineTee(design, 'tee'+ID, options=tee_options)

    spacing = 200
    L = int(freq2length(f)) - coupling_length - down_length

    short1_options = Dict(pos_x=str(short_pos[0])+'um', pos_y=str(short_pos[1])+'um', orientation='-90', termination_gap='10um')
    short1 = ShortToGround(design,'short'+ID,options=short1_options)

    #f = 5e9
    res1_options = Dict(total_length=str(L)+'um', fillet=str(-5+spacing/2)+'um', lead = dict(start_straight='100um', end_straight='0um'),
                        pin_inputs=Dict(start_pin=Dict(component='tee'+ID, pin='second_end'),
                                        end_pin=Dict(component='short'+ID, pin='short')),
                        meander=Dict(spacing=str(spacing)+'um', asymmetry='0um'),
                        trace_width=RES_WIDTH, trace_gap=res_gap)
    res1 = RouteMeander(design, 'res'+ID,  options=res1_options)

In [4]:
def render_qiskit_metal_design():
    simulation = False
    ### 7.25 GHz ###
    ID = '1'
    f = 7.25e9-0.175e9
    coupling_length = 300
    if simulation:
        prime_length = coupling_length+700
    else:
        prime_length = 0
    down_length = 200
    coupling_space = 70
    tee_pos = [-2500, 500]
    tee_fillet = 80
    right = True
    orientation = '90'
    res_gap = '30um'
    short_pos = [-1000, tee_pos[1]-coupling_length+200]

    resonator(ID, f, tee_pos, coupling_length, prime_length, coupling_space, short_pos, res_gap=res_gap, down_length = down_length, fillet = tee_fillet, right = right, orientation=orientation)

    gui.rebuild()
    gui.autoscale() 

In [5]:
render_qiskit_metal_design()

In [6]:
import names as n
import numpy as np
import parameter_targets as pt

from qdesignoptimizer.design_analysis_types import MiniStudy
from qdesignoptimizer.sim_capacitance_matrix import (
    CapacitanceMatrixStudy,
    ModeDecayIntoChargeLineStudy,
)
from qdesignoptimizer.utils.names_design_variables import junction_setup
from qdesignoptimizer.utils.names_parameters import FREQ, param

In [7]:
import mini_studies as ms
import optimization_targets as ot
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qdesignoptimizer.design_analysis_types import MeshingMap

In [8]:
from typing import List

import names as n
import numpy as np

from qdesignoptimizer.design_analysis_types import OptTarget
from qdesignoptimizer.utils.optimization_targets import (
    get_opt_targets_qb_res_transmission,
)

In [ ]:
def get_mini_study_resonator_capacitance(group: int):
    qiskit_component_names = ['res1', 'tee1']
    cap_study = CapacitanceMatrixStudy(
        qiskit_component_names=qiskit_component_names,
        open_pins=[
            ('tee1', "start"),
            ('tee1', "end"),
        ],
        mode_freq_GHz=7.25 * 1e-9,
        nbr_passes=8,
    )
    return MiniStudy(
        qiskit_component_names=qiskit_component_names,
        port_list=[],
        open_pins=[],
        modes=[],  # No mode frequencies to run only capacitance studies and not eigenmode/epr
        jj_setup={},
        design_name="get_mini_study_capacitance",
        adjustment_rate=1,
    )

In [ ]:
def get_opt_target_capacitance():

    return [
        OptTarget(
            target_param_type=n.CAPACITANCE,
            involved_modes=["prime_cpw_name_tee1", "second_cpw_name_tee1"],
            design_var=n.design_var_length(f"{resonator}_capacitance"),
            design_var_constraint={"larger_than": "1um", "smaller_than": "500um"},
            prop_to=lambda p, v: 1,
            independent_target=True,
        )
    ]

In [ ]:
MINI_STUDY_BRANCH = 1
MINI_STUDY = get_mini_study_resonator_capacitance(group=MINI_STUDY_BRANCH)
opt_targets = get_opt_target_capacitance()

RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)
# %matplotlib inline
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + 'CHIP_NAME' + "_",
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_CAPACITANCE,
)

In [ ]:
from qdesignoptimizer.utils.utils import close_ansys

In [ ]:
close_ansys()

In [9]:
cap_study = CapacitanceMatrixStudy(
        qiskit_component_names=['res1', 'tee1'],
        open_pins=[
            ('res1', 'end'),
            ('tee1', "prime_start"),
            ('tee1', "prime_end"),
        ],
        mode_freq_GHz=7.25 * 1e-9,
        nbr_passes=8,
    )

In [10]:
cap_study.simulate_capacitance_matrix(design)

INFO 03:13PM [connect_project]: Connecting to Ansys Desktop API...
INFO 03:13PM [load_ansys_project]: 	Opened Ansys App
INFO 03:13PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 03:13PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/lukassp/Documents/Ansoft/
	Project:   Project15
INFO 03:13PM [connect_design]: No active design found (or error getting active design).
INFO 03:13PM [connect]: 	 Connected to project "Project15". No design detected
INFO 03:13PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
WARNING 03:13PM [connect_setup]: 	No design setup detected.
WARNING 03:13PM [connect_setup]: 	Creating Q3D default setup.
INFO 03:13PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 03:13PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 03:13PM [analyze]: Analyzing setup Setup
INFO 03:13PM [get_matrix]: Exporting matrix data to (C:\Users\lukassp\AppData\Loc

,ground_main_plane,prime_cpw_tee1,second_cpw_tee1
ground_main_plane,820.96983,-91.94068,-503.09355
prime_cpw_tee1,-91.94068,92.89104,-0.89915
second_cpw_tee1,-503.09355,-0.89915,504.63575


In [11]:
def qc(imp, cap, omega):
    return np.pi / (imp**2 * omega**2 * cap**2)

In [14]:
freq= 7.25e9
2*freq/qc(60, 0.89915e-15, 2*np.pi*freq)/1e3

27.87537235915567